<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/projects/faq_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Шаблон проекта по теме "Чат-бот для частозадаваемых вопросов"

разработка бота с помощью инструментов NLP для FAQ какой-либо организации (например, НИУ ВШЭ), использовать запросы с большим языковым моделям в работе бота

Критерии оценивания

- Выгружена база знаний организации, для которой разрабатывается бот (например, произведен парсинг веб-страниц с информацией о НИУ ВШЭ) (2 балла)
- База знаний преобразована к виду векторной, реализована функция для поиска семантического сходства (2 балла)
- Выполнена функция для генерации ответов на вопросы по информации, извлеченной из базы знаний (2 балла)
- Как минимум один запрос успешно выполняется через Telegram-бот (2 балла)
- В *.ipynb приложены скриншоты, демонстрирующие работу бота (2 балла)

### Шаг 1

Используйте туториалы модуля 2 по веб-скрейпингу, чтобы выгрузить информацию с одной или нескольких страниц для формирования базы знаний вашего бота

Уделите внимание сегментации данных - что будет храниться в одном документе?

### Шаг 2

Преобразуйте ваш набор текстов к виду векторной базы данных по образцу

In [ ]:
!pip install sentence-transformers -q

In [ ]:
from sentence_transformers import SentenceTransformer

# Загружаем модель (энкодер Транфсормера для формирования векторного пространства)
model = SentenceTransformer('all-MiniLM-L6-v2')

# 1. Кодируем список текстов (документов) из переменной docs с помощью SentenceTransformer
embeddings = model.encode(docs, convert_to_numpy=True)

# 2. Создаем индекс FAISS (это модель для эмбеддингов)
dim = embeddings.shape[1]  # Рассчитываем размерность матрицы эмбеддингов
index = faiss.IndexFlatL2(dim)  # Создаем пустую векторную БД нужной размерности dim

# Добавляем  embeddings to FAISS index
index.add(embeddings)

# 3. Делаем запрос к БД: кодируем текущий запрос пользователя
embedding = model.encode([text], convert_to_numpy=True)

# Производим поиск по БД, ищем k ближайших по вектору документов
D, I = index.search(embedding, k=1)

# Здесь будет ближайшее по вектору совпадение
extracted_sample = docs[I]

### Шаг 3

Инициализируйте клиента HuggingFace API

Измените системный и пользовательский промпты так, чтобы получилась функция для генерации заданий в формате викторины по темам любой дисциплины на выбор (например, география или история)

Используйте образцы:

In [ ]:
# Инициализация клиента API
from huggingface_hub import InferenceClient

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
client = InferenceClient(model_name, token='ваш токен здесь')

In [ ]:
# Создание функции для запросов к модели
def llm_inference(extracted_sample):
  output = client.chat.completions.create(
          # Промпты
          messages=[
              # Основная системная инструкция
              {"role": "system", "content": "you are natural language processing guide\n"
                                            "explain the provided topic\n"
              },
              # Пользовательская инструкция: на место extracted_sample
              # помещаем информацию, извлеченную из базы знаний
              {"role": "user",
              "content": f"explain the basics of {extracted_sample}"},
          ],
          stream=False,
          max_tokens=128, # Максимальная длина вывода
          temperature=0.7, # Температура
          top_p=0.1 # Объем выборки для сэмплирования
          )
  # Вывод текста, ответа модели
  return output.choices[0].get('message')['content']

### Шаг 4

Используйте туториал по работе с aiogram и домашнее задание 3, чтобы разработать функционал Telegram-бота

1. Бот принимает на вход вопрос пользователя
2. Бот ищет в базе знаний наиболее релевантный ответ
3. Бот передает найденный документ языковой модели
4. Языковая модель генерирует результат, который выводится ботом

### Шаг 5

Вставьте в ячейку ниже скриншоты, которые демонстрируют работу Telegram-бота